# <center>firestoreにインポートするためのスクリプト</center>

In [1]:
# 使用するライブラリ
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
from google.cloud.firestore import GeoPoint

# firebaseに認証を行う
cred = credentials.Certificate('virtual-pilgrimage-dev-firebase-adminsdk-fnt9u-dbd50504a7.json')
app = firebase_admin.initialize_app(cred)
# firestoreのデータベースにアクセスする
db = firestore.client()

/Users/msz/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
db

In [3]:
# お寺の情報を読み込む
df = pd.read_csv("temple_info_update.csv",encoding="utf-8")

In [32]:
# データの確認
df.columns

Index(['Unnamed: 0', 'お遍路の番号', '名前', '名前（仮名）', '正式名称', '宗派', '本尊', '住所',
       '緯度(北緯)', '経度(東経)', '電話番号', '徒歩距離[km]', '車での距離[km](使用道路)', '駐車可能台数',
       '駐車場', '駐車場の位置', '車でのアクセス', 'JRでのアクセス', 'ホテル', 'トイレ', '徒歩距離(m)'],
      dtype='object')

### データの結びつけ
- address:住所
- distance:徒歩距離(m)
- geoPoint:[緯度(北緯),経度(東経)]
- id:お遍路の番号
- images:
- name:名前
- prefecture:住所[:3]

In [18]:
# 各種リストの作成
address = df["住所"].to_list()
distance = df["徒歩距離(m)"].to_list()
geo_n = df["緯度(北緯)"].to_list()
geo_t = df["経度(東経)"].to_list()
id_list = df["お遍路の番号"].to_list()
name = df["名前"].to_list()


In [14]:
# geopointの作成
geoPoint = []
for n,t in zip(geo_n,geo_t):
    geoPoint.append(GeoPoint(n,t))

In [15]:
prefecture = [add[:3] for add in address]

In [17]:
# データ数が同じかを確認する
print(len(address),len(distance),len(id_list),len(geoPoint),len(name),len(prefecture))

88 88 88 88 88 88


In [ ]:
# imageのパスの例
# https://firebasestorage.googleapis.com/v0/b/virtual-pilgrimage-dev.appspot.com/o/temples%2F1%2F1.jpg?alt=media&token=6fe4c2ed-b204-4837-9c56-3b72f24e1186

In [28]:
image_list = []
for id_ in id_list:
    image_path = "https://firebasestorage.googleapis.com/v0/b/virtual-pilgrimage-dev.appspot.com/o/temples%2F"+str(id_)+"%2F1.jpg?alt=media&token=6fe4c2ed-b204-4837-9c56-3b72f24e1186"
    image_list.append([image_path])

In [29]:
image_list

[['https://firebasestorage.googleapis.com/v0/b/virtual-pilgrimage-dev.appspot.com/o/temples%2F1%2F1.jpg?alt=media&token=6fe4c2ed-b204-4837-9c56-3b72f24e1186'],
 ['https://firebasestorage.googleapis.com/v0/b/virtual-pilgrimage-dev.appspot.com/o/temples%2F2%2F1.jpg?alt=media&token=6fe4c2ed-b204-4837-9c56-3b72f24e1186'],
 ['https://firebasestorage.googleapis.com/v0/b/virtual-pilgrimage-dev.appspot.com/o/temples%2F3%2F1.jpg?alt=media&token=6fe4c2ed-b204-4837-9c56-3b72f24e1186'],
 ['https://firebasestorage.googleapis.com/v0/b/virtual-pilgrimage-dev.appspot.com/o/temples%2F4%2F1.jpg?alt=media&token=6fe4c2ed-b204-4837-9c56-3b72f24e1186'],
 ['https://firebasestorage.googleapis.com/v0/b/virtual-pilgrimage-dev.appspot.com/o/temples%2F5%2F1.jpg?alt=media&token=6fe4c2ed-b204-4837-9c56-3b72f24e1186'],
 ['https://firebasestorage.googleapis.com/v0/b/virtual-pilgrimage-dev.appspot.com/o/temples%2F6%2F1.jpg?alt=media&token=6fe4c2ed-b204-4837-9c56-3b72f24e1186'],
 ['https://firebasestorage.googleapis.co

In [30]:
# データの書き込み
"""
データの結びつけ
- address:住所
- distance:徒歩距離(m)
- geoPoint:[緯度(北緯),経度(東経)]
- id:お遍路の番号
- images:
- name:名前
- prefecture:住所[:3]
"""
for i,id_ in enumerate(id_list):
    ref = db.collection(u'temples').document(str(id_))
    ref.set({
        u'address':address[i],
        u'distance':distance[i],
        u'geoPoint':geoPoint[i],
        u'id':id_,
        u'images':image_list[i],
        u'name':name[i],
        u'prefecture':prefecture[i]
    })